# 実験概要
## CMNによる特徴量抽出
**ケプストラム平均正規化** (Cepstrum Mean Normalization : CMN) を使用し、特徴量を抽出する。
切り出した波形に対して**離散フーリエ変換** (Discrete Fourier Transform : DFT) を行い、絶対値を取ることで**振幅スペクトル**を得る。
これに対して対数を取って**対数振幅スペクトル**に変換し、逆離散フーリエ変換 (Inverse Discrete Fourier Transform : IDFT) を行うことで、ケプストラム領域へと変換し、先頭50要素を切り出し、結合した100要素の配列を前処理として返す。
この前処理は
```
left, right, posture = pp.slicer("raw\\" + tester.**.**.value)
cepstrum = pp.cmn_denoise(left, right)
```
によって行われる。

## DNNによる寝姿勢分類
DNNによって前処理したデータから学習・分類を行う。

# モジュールのインポート

In [1]:
import numpy as np
import glob

# 自作モジュール
import datapath as dpath
import preprocess as pp

# DataLoaderの定義

In [2]:
# tester以外のrawを読み込む
for p in glob.glob(".\\raw\\LMH\\*\\", recursive=True):
    print(p)

In [3]:
e = dpath.LMH.H002.value.fl_center
print(e.value)

type, tester, mattress = dpath.getattributes(e, position=False)
print(f"type : {type},  tester : {tester},  mattress : {mattress}")
train_paths = eval(f"dpath.{type}.serch('{mattress}')")
print(train_paths)

raw\LMH\H002\fl_center
type : LMH,  tester : H002,  mattress : fl
[WindowsPath('raw/LMH/H002/fl_center'), WindowsPath('raw/LMH/H003/fl_center'), WindowsPath('raw/LMH/L001/fl_center'), WindowsPath('raw/LMH/L003/fl_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M001/fld_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M002/fls_center'), WindowsPath('raw/LMH/M002/fld_center'), WindowsPath('raw/LMH/M002/fls_center'), WindowsPath('raw/LMH/M003/fls_center'), WindowsPath('raw/LMH/M003/fld_center'), WindowsPath('raw/LMH/M003/fls_center'), WindowsPath('raw/LMH/M004/fls_center'), WindowsPath('raw/LMH/M004/fld_center'), WindowsPath('raw/LMH/M004/fls_center'), WindowsPath('raw/LMH/H002/fl_center'), WindowsPath('raw/LMH/H003/fl_center'), WindowsPath('raw/LMH/L001/fl_center'), WindowsPath('raw/LMH/L003/fl_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/M001/fld_center'), WindowsPath('raw/LMH/M001/fls_center'), WindowsPath('raw/LMH/

In [4]:
import torch
from typing import Tuple
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 学習用データセットを読み込むためのDataLoaderを定義
class train_datasets(Dataset):
    def __init__(self, identity, transforms=None):
        self.transform = transforms
        self.type, self.tester, self.mattress = dpath.getattributes(identity)
        print(self.type, self.tester, self.mattress)
        print("train")

        # テスト以外のrawを読み込む
        self.train_cepstrum = np.empty((0, 100))  # 100要素の配列
        self.train_posture = np.empty(0)  # 姿勢データ配列

        # mattressに該当するrawのpathを読み込む
        train_paths = eval(f"dpath.{self.type}.serch('{self.mattress}')")
        for p in train_paths:
            if identity.value == p:
                continue
            left, right, posture = pp.slicer(p)
            cepstrum = pp.cmn_denoise(left, right)
            for c in cepstrum:
                self.train_cepstrum = np.vstack((self.train_cepstrum, c)) if self.train_cepstrum.size else c
            self.train_posture = np.append(self.train_posture, posture) if self.train_posture.size else posture

    def __len__(self):
        return len(self.train_posture)

    def __getitem__(self, idx) -> Tuple[torch.tensor, torch.tensor]:
        cepstrum = torch.tensor(self.train_cepstrum[idx].reshape(1, -1), dtype=torch.float32)
        posture = torch.tensor(self.train_posture[idx]-1, dtype=torch.long)
        if self.transform:
            cepstrum = self.transform(cepstrum)
        return cepstrum, posture


# test用データセット
class test_datasets(Dataset):
    def __init__(self, identity, transform=None):
        self.transform = transform
        self.type, self.tester, self.mattress = dpath.getattributes(identity)
        print("test")

        # 前処理したrawを読み込む
        self.left, self.right, self.test_posture = pp.slicer(identity.value)
        self.test_cepstrum = pp.cmn_denoise(self.left, self.right)

    def __len__(self):
        return len(self.test_posture)

    def __getitem__(self, idx) -> Tuple[torch.tensor, torch.tensor]:
        cepstrum = torch.tensor(self.test_cepstrum[idx].reshape(1, -1), dtype=torch.float32)
        posture = torch.tensor(self.test_posture[idx]-1, dtype=torch.long)
        if self.transform:
            cepstrum = self.transform(cepstrum)
        return cepstrum, posture

# 学習＆検証

In [5]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [6]:
identity = dpath.LMH.H002.value.st_center

train = train_datasets(identity)
test = test_datasets(identity)
batch_size = 128

train_loader = DataLoader(
    train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

test_loader = DataLoader(
    test,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

LMH H002 st
train
data[340 / 403]
data[186 / 208]
data[173 / 208]
data[192 / 324]
data[174 / 438]
data[220 / 481]
data[87 / 281]
data[340 / 403]
data[186 / 208]
data[173 / 208]
data[192 / 324]
data[174 / 438]
data[220 / 481]
data[87 / 281]
test
data[194 / 230]


## 学習

In [11]:
import torch.optim as optim
import torch.nn as nn
import model_resnet as resnet

# モデルのインスタンス化
net = resnet.resnet18_1d(num_classes=4).to(device)

# 誤差関数を交差エントロピーで計算
criterion = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.SGD(net.parameters(), lr = 1e-6, momentum=0.9)

# 学習
n_epoch = 100
for epoch in range(n_epoch):
    # 精度と損失の初期化
    train_acc, train_loss = 0, 0
    val_acc, val_loss = 0, 0
    n_train, n_test = 0, 0

    # 学習
    for train_input, train_label in train_loader:
        n_train += len(train_label)

        # 入力と正解ラベルをGPU上に移動
        input = train_input.to(device)
        label = train_label.to(device)
        # print(f'input : {input.shape}, label : {label.shape}')

        optimizer.zero_grad()
        output = net(input)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        predicted = torch.max(output, 1)[1]

        train_loss += loss.item()
        train_acc += (predicted == label).sum().item()

    # 検証
    for test_input, test_label in test_loader:
        n_test += len(test_label)

        test_input = test_input.to(device)
        test_label = test_label.to(device)

        test_output = net(test_input)
        test_loss = criterion(test_output, test_label)

        test_predicted = torch.max(test_output, 1)[1]

        val_loss += test_loss.item()
        val_acc += (test_predicted == test_label).sum().item()

    # 精度を確率に変換
    train_acc /= n_train
    val_acc /= n_test
    train_loss = train_loss * batch_size / n_train
    val_loss = val_loss * batch_size / n_test

    print(f"Epoch[{epoch+1}/{n_epoch}], | loss: {train_loss:.5f} | acc: {train_acc:.5f} | val_loss: {val_loss:.5f} | val_acc: {val_acc:.5f}")


Epoch[1/100], | loss: 1.38327 | acc: 0.36161 | val_loss: 1.51705 | val_acc: 0.29688
Epoch[2/100], | loss: 1.34520 | acc: 0.40439 | val_loss: 1.47818 | val_acc: 0.35156
Epoch[3/100], | loss: 1.30463 | acc: 0.45871 | val_loss: 1.51318 | val_acc: 0.34375
Epoch[4/100], | loss: 1.26898 | acc: 0.49665 | val_loss: 1.43020 | val_acc: 0.34375
Epoch[5/100], | loss: 1.23914 | acc: 0.53162 | val_loss: 1.41541 | val_acc: 0.41406
Epoch[6/100], | loss: 1.21203 | acc: 0.56473 | val_loss: 1.40559 | val_acc: 0.44531
Epoch[7/100], | loss: 1.18701 | acc: 0.59821 | val_loss: 1.39980 | val_acc: 0.40625
Epoch[8/100], | loss: 1.16354 | acc: 0.63170 | val_loss: 1.41383 | val_acc: 0.42969
Epoch[9/100], | loss: 1.14323 | acc: 0.64658 | val_loss: 1.37008 | val_acc: 0.52344
Epoch[10/100], | loss: 1.12407 | acc: 0.67374 | val_loss: 1.36144 | val_acc: 0.50781
Epoch[11/100], | loss: 1.10614 | acc: 0.69606 | val_loss: 1.36206 | val_acc: 0.48438
Epoch[12/100], | loss: 1.08709 | acc: 0.72135 | val_loss: 1.33644 | val_ac